In [1]:
from FunctionSet import *
# The NRPy_param_funcs module sets up global structures that manage free parameters within NRPy+
import NRPy_param_funcs as par 
# The indexedexp module defines various functions for defining and managing indexed quantities like tensors and pseudotensors
import indexedexp as ixp
# The grid module defines various parameters related to a numerical grid or the dimensionality of indexed expressions
# For example, it declares the parameter DIM, which specifies the dimensionality of the indexed expression
import grid as gri
from outputC import *
import numpy as np

import sympy as sp
from sympy import symbols, IndexedBase, Indexed, Idx, preorder_traversal
from sympy.matrices import *

Declaring indexed variables.

In [2]:
path = "../Source/ADM/"

In [3]:
Nx, Ny, Nz= symbols('Nx Ny Nz', integer=True)
i = Idx('i', Nx)
j = Idx('j', Ny)
k = Idx('k', Nz)
name = Idx('name')
dx, dy, dz = symbols('dx dy dz')

Declaring variables and types.  Will need to find a more efficient way to do this.  It works for now.|

In [4]:
var_alpha = ['alpha']

alpha = IndexedBase('state_fab'+var_alpha[0])[i,j,k,var_alpha[0]]

var_beta = ['betax','betay','betaz']

betaL = ixp.zerorank1(DIM=3)

for itr in range(len(directions)):
    betaL[itr] = IndexedBase('state_fab'+var_beta[itr])[i,j,k,var_beta[itr]]
    
betaL = np.array(betaL)
    
var_gam = [['gxx','gxy','gxz'],['gyx','gyy','gyz'],['gzx','gzy','gzz']]

gamLL = ixp.declarerank2('g','sym01')
for itr1 in range(len(directions)):
    for itr2 in range(len(directions)):
        gamLL[itr1][itr2] = IndexedBase('state_fab'+var_gam[itr1][itr2])[i,j,k,var_gam[itr1][itr2]]

gamLL = np.array(gamLL)

var_gaminv = [['ginvxx','ginvxy','ginvxz'],['ginvyx','ginvyy','ginvyz'],['ginvzx','ginvzy','ginvzz']]
        
gamUU = ixp.declarerank2('ginv','sym01')
for itr1 in range(len(directions)):
    for itr2 in range(len(directions)):
        gamUU[itr1][itr2] = IndexedBase('state_fab'+var_gaminv[itr1][itr2])[i,j,k,var_gaminv[itr1][itr2]]
        
gamUU = np.array(gamUU)
        
var_K = [['Kxx','Kxy','Kxz'],['Kyx','Kyy','Kyz'],['Kzx','Kzy','Kzz'],]
        
KLL = ixp.declarerank2('K','sym01')
for itr1 in range(len(directions)):
    for itr2 in range(len(directions)):
        KLL[itr1][itr2] = IndexedBase('state_fab'+var_K[itr1][itr2])[i,j,k,var_K[itr1][itr2]]

KLL = np.array(KLL)

In [5]:
var_beta

['betax', 'betay', 'betaz']

Making a list of all variable names to assign string replacements in AMReXcode output later.

In [6]:
var_names=[]
for itr in range(len(var_beta)):
    var_names.append(var_beta[itr])
print(var_names)

['betax', 'betay', 'betaz']


In [7]:
var_names = []
for itr in range(len(var_beta)):
    var_names.append(var_beta[itr])
for iter1 in var_gam:
    var_names.extend(iter1)
for iter2 in var_gaminv:
    var_names.extend(iter2)
for iter3 in var_K:
    var_names.extend(iter3)
var_names.insert(0,var_alpha[0])
print(var_names)

['alpha', 'betax', 'betay', 'betaz', 'gxx', 'gxy', 'gxz', 'gyx', 'gyy', 'gyz', 'gzx', 'gzy', 'gzz', 'ginvxx', 'ginvxy', 'ginvxz', 'ginvyx', 'ginvyy', 'ginvyz', 'ginvzx', 'ginvzy', 'ginvzz', 'Kxx', 'Kxy', 'Kxz', 'Kyx', 'Kyy', 'Kyz', 'Kzx', 'Kzy', 'Kzz']


Constructing the Gamma{i,j,k} connection coefficients from the metric.

In [9]:
DcgLLL = ixp.zerorank3(3)
GammaULL = ixp.zerorank3(3)

for itri in range(len(directions)):
    for itrj in range(len(directions)):
        for itrk in range(len(directions)):
            DcgLLL[itri][itrj][itrk] = Dc(gamLL[itri][itrj],directions[itrk])
            
for itri in range(len(directions)):
    for itrj in range(len(directions)):
        for itrk in range(len(directions)):
            for itrl in range(len(directions)):
                GammaULL[itri][itrj][itrk] += 1/2*gamUU[itri][itrl]*(DcgLLL[itrj][itrl][itrk]+DcgLLL[itrk][itrl][itrj]-DcgLLL[itrj][itrk][itrl])

Constructing covariant derivatives of beta.

In [10]:
covbetaLL = ixp.zerorank2(3)
covbetaTransLL = ixp.zerorank2(3)
for itri in range(len(directions)):
    for itrj in range(len(directions)):
        covbetaLL[itri][itrj] = Dc(betaL[itrj],directions[itri])
        covbetaTansLL = Dc(betaL[itri],directions[itrj])
        for itrk in range(len(directions)):
            covbetaLL[itri][itrj] += GammaULL[itrk][itri][itrj]*betaL[itrk]
            covbetaTransLL[itri][itrj] += GammaULL[itrk][itrj][itri]*betaL[itrk]

Constructing the RHS of metric in ADM evolution equations.  Evolution eq for K to come later.

In [11]:
gam_RHS = -2*alpha*KLL+covbetaLL+covbetaTransLL

alpha_Init = 0
beta_Init = np.array([0,0,0])
gam_Init = np.array([[1,0,0],[0,1,0],[0,0,1]])

In [12]:
var_beta

['betax', 'betay', 'betaz']

In [13]:
AMReXcode(beta_Init[0],var_names,declare_state=True, statename=var_beta[0])

'state_fab(i, j, k, Idx::betax) = 0;'

Create Setup.H, Variable.H, Init_K.H, and Rhs_K.H

In [14]:
createSETUP(path+"ET_Integration_Setup.H",var_names,1)

In [15]:
createVARIABLES(path+"ET_Integration_Variables.H",var_names)

In [16]:
createINIT(path+"ET_Integration_Init_K.H")
addINIT(path+"ET_Integration_Init_K.H",AMReXcode(alpha_Init,var_names,declare_state=True, statename=var_alpha[0]))
for itr in range(3):
    addINIT(path+"ET_Integration_Init_K.H",AMReXcode(beta_Init[itr],var_names,declare_state=True, statename=var_beta[itr]))
for itri in range(3):
        for itrj in range(3):
            addINIT(path+"ET_Integration_Init_K.H",AMReXcode(gam_Init[itri][itrj],var_names,declare_state=True, statename=var_gam[itri][itrj]))
finishINIT(path+"ET_Integration_Init_K.H")

In [17]:
createRHS(path+"ET_Integration_Rhs_K.H")
for itri in range(3):
        for itrj in range(3):
            addRHS(path+"ET_Integration_Rhs_K.H",AMReXcode(gam_RHS[itri][itrj],var_names,declare_rhs=True, rhsname=var_gam[itri][itrj]))
finishRHS(path+"ET_Integration_Rhs_K.H")